In [206]:
import json
import pandas as pd
import numpy as np
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, linregress
import re
sns.set_style('whitegrid')
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 100)

In [14]:
with open('data1.json') as data_file:    
    data = json.load(data_file)

In [15]:
for i, j in data.items():
    pprint(i)

'player'
'phases'
'element_types'
'current-event'
'game-settings'
'stats_options'
'entry'
'next-event'
'next_event_fixtures'
'last-entry-event'
'teams'
'elements'
'events'
'total-players'
'stats'
'watched'


In [185]:
data['game-settings']

{'element_type': {'1': {'bps_clean_sheets': 12,
   'bps_goals_scored': 12,
   'scoring_clean_sheets': 4,
   'scoring_goals_conceded': -1,
   'scoring_goals_scored': 6,
   'squad_max_play': 1,
   'squad_min_play': 1,
   'squad_select': 2,
   'sub_positions_locked': [12],
   'ui_shirt_specific': True},
  '2': {'bps_clean_sheets': 12,
   'bps_goals_scored': 12,
   'scoring_clean_sheets': 4,
   'scoring_goals_conceded': -1,
   'scoring_goals_scored': 6,
   'squad_max_play': 5,
   'squad_min_play': 3,
   'squad_select': 5},
  '3': {'bps_clean_sheets': 0,
   'bps_goals_scored': 18,
   'scoring_clean_sheets': 1,
   'scoring_goals_conceded': 0,
   'scoring_goals_scored': 5,
   'squad_max_play': 5,
   'squad_min_play': 2,
   'squad_select': 5},
  '4': {'bps_clean_sheets': 0,
   'bps_goals_scored': 24,
   'scoring_clean_sheets': 0,
   'scoring_goals_conceded': 0,
   'scoring_goals_scored': 4,
   'squad_max_play': 3,
   'squad_min_play': 1,
   'squad_select': 3}},
 'game': {'bps_assists': 9,
  'b

In [30]:
player_data_json = data['elements']

In [31]:
player_data_df = pd.DataFrame(player_data_json)

In [126]:
pdata = player_data_df.copy()

In [127]:
pdata.head(3)

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,dreamteam_count,ea_index,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,id,in_dreamteam,influence,loaned_in,loaned_out,loans_in,loans_out,minutes,news,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,0,0,18,NaN,None,0,48844,0,0,0,0,0.0,0,0,1,3.4,None,0,David,0.0,4,0,2.9,1,False,28.8,0,0,0,0,143,,50,0,0,0,48844.jpg,1.0,0,4,Ospina,0.2,False,13.0,a,1,3,0.0,2,0,0,0,0,0.0,0.0,Ospina,0
1,0,2,660,NaN,None,12,11334,0,0,0,0,0.1,0,0,1,4.2,None,0,Petr,0.0,37,0,82.0,2,False,819.8,0,0,0,0,3097,,55,1,0,0,11334.jpg,3.8,0,115,Cech,5.6,False,33.0,a,1,3,0.0,134,0,0,0,0,0.0,0.0,Cech,2
2,0,0,35,0.0,None,1,98980,0,0,0,0,0.0,0,0,1,0.0,None,0,Damian Emiliano,0.0,3,0,3.1,3,False,30.6,0,0,0,0,180,Season-long loan to Getafe,40,0,0,0,98980.jpg,4.0,0,5,Martinez,1.3,False,26.0,u,1,3,0.0,8,0,0,0,0,0.0,0.0,Martinez,0


In [128]:
to_drop = ['chance_of_playing_this_round','chance_of_playing_next_round','code','cost_change_event','cost_change_event_fall','cost_change_start','cost_change_start_fall','dreamteam_count','ea_index','ep_this','event_points','form','ict_index','in_dreamteam','loaned_in','loaned_out','loans_in','loans_out','news','photo','special','squad_number','status','transfers_in','transfers_in_event','transfers_out','transfers_out_event','value_form','value_season']

In [129]:
pdata.drop(to_drop, axis=1, inplace = True)

In [130]:
pdata.columns

Index(['assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element_type',
       'ep_next', 'first_name', 'goals_conceded', 'goals_scored', 'id',
       'influence', 'minutes', 'now_cost', 'own_goals', 'penalties_missed',
       'penalties_saved', 'points_per_game', 'red_cards', 'saves',
       'second_name', 'selected_by_percent', 'team', 'team_code', 'threat',
       'total_points', 'web_name', 'yellow_cards'],
      dtype='object')

In [131]:
pdata['full_name'] = pdata.first_name + " " + pdata.second_name

In [132]:
pdata['element_type_name'] = pdata.element_type.map({x['id']:x['singular_name_short'] for x in data['element_types']})

In [133]:
pdata = pdata.loc[:,['full_name','first_name','second_name', 'element_type','element_type_name','id','team', 'team_code', 'web_name',
                     'saves','penalties_saved','clean_sheets','goals_conceded',
                     'bonus', 'bps','creativity','ep_next','influence', 'threat',
                     'goals_scored','assists','minutes', 'own_goals',
                     'yellow_cards', 'red_cards','penalties_missed',
                     'selected_by_percent', 'now_cost','points_per_game','total_points']]

In [134]:
pdata['team'] = pdata.team.map({x['id']:x['name'] for x in data['teams']})

In [147]:
pdata.head()

,full_name,first_name,second_name,element_type,element_type_name,id,team,team_code,web_name,saves,penalties_saved,clean_sheets,goals_conceded,bonus,bps,creativity,ep_next,influence,threat,goals_scored,assists,minutes,own_goals,yellow_cards,red_cards,penalties_missed,selected_by_percent,now_cost,points_per_game,total_points
0,David Ospina,David,Ospina,1,GKP,1,Arsenal,3,Ospina,4,0,0,4,0,18,0.0,3.4,28.8,0.0,0,0,143,0,0,0,0,0.2,50,1.0,2
1,Petr Cech,Petr,Cech,1,GKP,2,Arsenal,3,Cech,115,0,12,37,2,660,0.1,4.2,819.8,0.0,0,0,3097,1,2,0,0,5.6,55,3.8,134
2,Damian Emiliano Martinez,Damian Emiliano,Martinez,1,GKP,3,Arsenal,3,Martinez,5,0,1,3,0,35,0.0,0.0,30.6,0.0,0,0,180,0,0,0,0,1.3,40,4.0,8
3,Laurent Koscielny,Laurent,Koscielny,2,DEF,4,Arsenal,3,Koscielny,0,0,10,37,19,723,53.9,0.0,831.2,239.0,2,1,2821,0,4,1,0,0.6,60,3.7,121
4,Per Mertesacker,Per,Mertesacker,2,DEF,5,Arsenal,3,Mertesacker,0,0,0,1,0,5,0.1,2.1,17.8,0.0,0,0,38,0,0,0,0,0.6,50,1.0,1


In [148]:
pdata.corr()['total_points']

element_type        0.040511
id                 -0.087756
team_code          -0.058783
saves               0.213275
penalties_saved     0.113341
clean_sheets        0.889073
goals_conceded      0.753940
bonus               0.875165
bps                 0.940472
goals_scored        0.711255
assists             0.703371
minutes             0.900990
own_goals           0.103359
yellow_cards        0.545764
red_cards           0.129938
penalties_missed    0.292188
now_cost            0.592595
total_points        1.000000
Name: total_points, dtype: float64

In [184]:
pdata.pivot_table(index='element_type_name', values='total_points', aggfunc=np.mean)

element_type_name
DEF    51.744048
FWD    58.542857
GKP    51.229167
MID    54.181395
Name: total_points, dtype: float64

In [178]:
pdata.element_type_name.value_counts()

MID    215
DEF    168
FWD     70
GKP     48
Name: element_type_name, dtype: int64

In [174]:
pdata.columns

Index(['full_name', 'first_name', 'second_name', 'element_type',
       'element_type_name', 'id', 'team', 'team_code', 'web_name', 'saves',
       'penalties_saved', 'clean_sheets', 'goals_conceded', 'bonus', 'bps',
       'creativity', 'ep_next', 'influence', 'threat', 'goals_scored',
       'assists', 'minutes', 'own_goals', 'yellow_cards', 'red_cards',
       'penalties_missed', 'selected_by_percent', 'now_cost',
       'points_per_game', 'total_points'],
      dtype='object')

In [155]:
pdata.element_type_name.value_counts()

MID    215
DEF    168
FWD     70
GKP     48
Name: element_type_name, dtype: int64

# Pulp implementation

In [214]:
from pulp import *

In [215]:
prob = pulp.LpProblem('FantasyTeam', pulp.LpMaximize)

In [216]:
decision_variables = []
for rownum, row in pdata.iterrows():
    variable = str('x' + str(rownum))
    variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer') #make variables binary
    decision_variables.append(variable)

print ("Total number of decision_variables: " + str(len(decision_variables)))

Total number of decision_variables: 501


In [217]:
total_points = ""
for rownum, row in pdata.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            formula = row['total_points']*schedule
            total_points += formula

prob += total_points
print ("Optimization function: " + str(total_points))

Optimization function: 2*x0 + 134*x1 + 32*x10 + 132*x100 + 105*x101 + 224*x103 + 121*x104 + 114*x105 + 105*x106 + 6*x107 + 162*x108 + 3*x109 + 90*x11 + 83*x110 + 12*x111 + 48*x112 + 196*x113 + 92*x114 + 75*x116 + 61*x117 + 86*x118 + 46*x119 + 51*x120 + 6*x121 + 47*x122 + 98*x123 + 92*x124 + 149*x125 + 93*x126 + 24*x127 + 91*x128 + 7*x129 + 264*x13 + 116*x130 + 42*x131 + 18*x132 + 136*x133 + 102*x134 + 89*x135 + 68*x136 + 135*x137 + 133*x138 + 104*x139 + 122*x14 + 9*x140 + 69*x141 + 127*x142 + 54*x143 + x144 + 113*x145 + 45*x146 + 119*x147 + 144*x148 + 23*x149 + 167*x15 + 68*x150 + 22*x152 + 37*x153 + 80*x154 + 63*x155 + 16*x156 + 17*x158 + 38*x16 + 76*x160 + 56*x17 + 83*x18 + 109*x188 + 52*x19 + 63*x190 + 104*x191 + 85*x192 + 88*x193 + 27*x194 + 19*x195 + 71*x197 + 120*x198 + 95*x199 + 8*x2 + 89*x20 + 41*x200 + 69*x201 + 58*x202 + 53*x203 + 6*x204 + 59*x205 + 39*x206 + 161*x208 + 24*x209 + 26*x21 + 63*x210 + 83*x211 + 110*x212 + 32*x213 + 106*x214 + 15*x215 + 129*x216 + 97*x218 + 43*x2

In [218]:
avail_cash = 1000
total_paid = ""
for rownum, row in pdata.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            formula = row['now_cost']*schedule
            total_paid += formula

prob += (total_paid == avail_cash)

In [219]:
avail_gk = 2
total_gk = ""
for rownum, row in pdata.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            if row['element_type_name'] == 'GKP':
                formula = 1*schedule
                total_gk += formula

prob += (total_gk == avail_gk)
print(total_gk)

x0 + x1 + x114 + x115 + x134 + x135 + x136 + x161 + x162 + x188 + x189 + x2 + x212 + x213 + x239 + x240 + x259 + x260 + x261 + x28 + x285 + x286 + x29 + x30 + x304 + x305 + x327 + x328 + x349 + x350 + x374 + x375 + x395 + x396 + x419 + x420 + x437 + x438 + x468 + x472 + x478 + x52 + x53 + x73 + x74 + x93 + x94 + x95


In [220]:
avail_def = 5
total_def = ""
for rownum, row in pdata.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            if row['element_type_name'] == 'DEF':
                formula = 1*schedule
                total_def += formula

prob += (total_def == avail_def)
print(len(total_def))

168


In [221]:
avail_mid = 5
total_mid = ""
for rownum, row in pdata.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            if row['element_type_name'] == 'MID':
                formula = 1*schedule
                total_mid += formula

prob += (total_mid == avail_mid)
print(len(total_mid))

215


In [222]:
avail_fwd = 3
total_fwd = ""
for rownum, row in pdata.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            if row['element_type_name'] == 'FWD':
                formula = 1*schedule
                total_fwd += formula

prob += (total_fwd == avail_fwd)
print(len(total_fwd))

70


In [223]:
print(prob)

FantasyTeam:
MAXIMIZE
2*x0 + 134*x1 + 32*x10 + 132*x100 + 105*x101 + 224*x103 + 121*x104 + 114*x105 + 105*x106 + 6*x107 + 162*x108 + 3*x109 + 90*x11 + 83*x110 + 12*x111 + 48*x112 + 196*x113 + 92*x114 + 75*x116 + 61*x117 + 86*x118 + 46*x119 + 51*x120 + 6*x121 + 47*x122 + 98*x123 + 92*x124 + 149*x125 + 93*x126 + 24*x127 + 91*x128 + 7*x129 + 264*x13 + 116*x130 + 42*x131 + 18*x132 + 136*x133 + 102*x134 + 89*x135 + 68*x136 + 135*x137 + 133*x138 + 104*x139 + 122*x14 + 9*x140 + 69*x141 + 127*x142 + 54*x143 + 1*x144 + 113*x145 + 45*x146 + 119*x147 + 144*x148 + 23*x149 + 167*x15 + 68*x150 + 22*x152 + 37*x153 + 80*x154 + 63*x155 + 16*x156 + 17*x158 + 38*x16 + 76*x160 + 56*x17 + 83*x18 + 109*x188 + 52*x19 + 63*x190 + 104*x191 + 85*x192 + 88*x193 + 27*x194 + 19*x195 + 71*x197 + 120*x198 + 95*x199 + 8*x2 + 89*x20 + 41*x200 + 69*x201 + 58*x202 + 53*x203 + 6*x204 + 59*x205 + 39*x206 + 161*x208 + 24*x209 + 26*x21 + 63*x210 + 83*x211 + 110*x212 + 32*x213 + 106*x214 + 15*x215 + 129*x216 + 97*x218 + 43*x

In [224]:
prob.writeLP('FantasyTeam.lp')

In [225]:
optimization_result = prob.solve()

In [226]:
assert optimization_result == pulp.LpStatusOptimal
print("Status:", LpStatus[prob.status])
print("Optimal Solution to the problem: ", value(prob.objective))
print ("Individual decision_variables: ")
for v in prob.variables():
	print(v.name, "=", v.varValue)

Status: Optimal
Optimal Solution to the problem:  2422.0
Individual decision_variables: 
x0 = 0.0
x1 = 0.0
x10 = 0.0
x100 = 0.0
x101 = 0.0
x102 = 0.0
x103 = 0.0
x104 = 0.0
x105 = 0.0
x106 = 0.0
x107 = 0.0
x108 = 0.0
x109 = 0.0
x11 = 0.0
x110 = 0.0
x111 = 0.0
x112 = 0.0
x113 = 0.0
x114 = 0.0
x115 = 0.0
x116 = 0.0
x117 = 0.0
x118 = 0.0
x119 = 0.0
x12 = 0.0
x120 = 0.0
x121 = 0.0
x122 = 0.0
x123 = 0.0
x124 = 0.0
x125 = 0.0
x126 = 0.0
x127 = 0.0
x128 = 0.0
x129 = 0.0
x13 = 0.0
x130 = 0.0
x131 = 0.0
x132 = 0.0
x133 = 0.0
x134 = 0.0
x135 = 0.0
x136 = 0.0
x137 = 0.0
x138 = 0.0
x139 = 0.0
x14 = 0.0
x140 = 0.0
x141 = 0.0
x142 = 0.0
x143 = 0.0
x144 = 0.0
x145 = 0.0
x146 = 0.0
x147 = 0.0
x148 = 0.0
x149 = 0.0
x15 = 0.0
x150 = 0.0
x151 = 0.0
x152 = 0.0
x153 = 0.0
x154 = 0.0
x155 = 0.0
x156 = 0.0
x157 = 0.0
x158 = 0.0
x159 = 0.0
x16 = 0.0
x160 = 0.0
x161 = 0.0
x162 = 0.0
x163 = 0.0
x164 = 0.0
x165 = 0.0
x166 = 0.0
x167 = 0.0
x168 = 0.0
x169 = 0.0
x17 = 0.0
x170 = 0.0
x171 = 0.0
x172 = 0.0
x173 = 0.0

In [227]:
variable_name = []
variable_value = []

for v in prob.variables():
    variable_name.append(v.name)
    variable_value.append(v.varValue)

df = pd.DataFrame({'variable': variable_name, 'value': variable_value})
for rownum, row in df.iterrows():
    value = re.findall(r'(\d+)', row['variable'])
    df.loc[rownum, 'variable'] = int(value[0])

df = df.sort_index(by='variable')

#append results
for rownum, row in pdata.iterrows():
    for results_rownum, results_row in df.iterrows():
        if rownum == results_row['variable']:
            pdata.loc[rownum, 'decision'] = results_row['value']
            
pdata.head()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:13: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


,full_name,first_name,second_name,element_type,element_type_name,id,team,team_code,web_name,saves,penalties_saved,clean_sheets,goals_conceded,bonus,bps,creativity,ep_next,influence,threat,goals_scored,assists,minutes,own_goals,yellow_cards,red_cards,penalties_missed,selected_by_percent,now_cost,points_per_game,total_points,decision
0,David Ospina,David,Ospina,1,GKP,1,Arsenal,3,Ospina,4,0,0,4,0,18,0.0,3.4,28.8,0.0,0,0,143,0,0,0,0,0.2,50,1.0,2,0.0
1,Petr Cech,Petr,Cech,1,GKP,2,Arsenal,3,Cech,115,0,12,37,2,660,0.1,4.2,819.8,0.0,0,0,3097,1,2,0,0,5.6,55,3.8,134,0.0
2,Damian Emiliano Martinez,Damian Emiliano,Martinez,1,GKP,3,Arsenal,3,Martinez,5,0,1,3,0,35,0.0,0.0,30.6,0.0,0,0,180,0,0,0,0,1.3,40,4.0,8,0.0
3,Laurent Koscielny,Laurent,Koscielny,2,DEF,4,Arsenal,3,Koscielny,0,0,10,37,19,723,53.9,0.0,831.2,239.0,2,1,2821,0,4,1,0,0.6,60,3.7,121,0.0
4,Per Mertesacker,Per,Mertesacker,2,DEF,5,Arsenal,3,Mertesacker,0,0,0,1,0,5,0.1,2.1,17.8,0.0,0,0,38,0,0,0,0,0.6,50,1.0,1,0.0


In [230]:
pdata[pdata.decision==1].sort_values('element_type')

,full_name,first_name,second_name,element_type,element_type_name,id,team,team_code,web_name,saves,penalties_saved,clean_sheets,goals_conceded,bonus,bps,creativity,ep_next,influence,threat,goals_scored,assists,minutes,own_goals,yellow_cards,red_cards,penalties_missed,selected_by_percent,now_cost,points_per_game,total_points,decision
73,Tom Heaton,Tom,Heaton,1,GKP,74,Burnley,90,Heaton,142,1,10,48,21,741,30.3,1.4,1001.6,0.0,0,0,3150,0,1,0,0,10.4,50,4.3,149,1.0
305,Fraser Forster,Fraser,Forster,1,GKP,306,Southampton,20,Forster,76,1,14,48,4,625,10.0,3.4,553.6,0.0,0,0,3420,0,3,0,0,7.1,50,3.5,134,1.0
33,Charlie Daniels,Charlie,Daniels,2,DEF,34,Bournemouth,91,Daniels,0,0,10,53,14,692,528.3,2.3,698.2,302.0,4,3,3060,0,1,0,0,22.0,50,3.9,134,1.0
96,CÃ©sar Azpilicueta,CÃ©sar,Azpilicueta,2,DEF,97,Chelsea,8,Azpilicueta,0,0,16,33,22,872,422.8,4.9,792.8,150.0,1,5,3420,0,4,0,0,11.5,65,4.5,170,1.0
97,Gary Cahill,Gary,Cahill,2,DEF,98,Chelsea,8,Cahill,0,0,17,29,15,778,100.8,4.9,831.6,400.0,6,0,3296,1,5,0,0,15.8,65,4.8,178,1.0
99,Marcos Alonso,Marcos,Alonso,2,DEF,100,Chelsea,8,Alonso,0,0,15,19,11,675,420.4,5.3,618.2,536.0,6,5,2693,0,2,0,0,13.6,70,5.7,177,1.0
424,Gareth McAuley,Gareth,McAuley,2,DEF,425,West Brom,35,McAuley,0,0,7,45,14,576,74.3,0.0,913.8,367.0,6,1,3140,1,5,0,0,3.0,50,3.6,131,1.0
336,Joe Allen,Joe,Allen,3,MID,337,Stoke,110,Allen,0,0,10,49,10,525,539.8,1.0,576.8,675.0,6,3,2930,0,9,0,0,5.1,55,3.3,118,1.0
386,Christian Eriksen,Christian,Eriksen,3,MID,387,Spurs,6,Eriksen,0,0,16,24,30,798,1679.2,5.0,974.6,1244.0,8,20,3159,0,0,0,0,14.8,95,6.1,218,1.0
387,Bamidele Alli,Bamidele,Alli,3,MID,388,Spurs,6,Alli,0,0,17,22,17,679,778.7,5.0,969.6,1270.0,18,11,3036,0,4,0,0,38.9,95,6.1,225,1.0


In [233]:
pdata.sort_values('now_cost',ascending=True)

,full_name,first_name,second_name,element_type,element_type_name,id,team,team_code,web_name,saves,penalties_saved,clean_sheets,goals_conceded,bonus,bps,creativity,ep_next,influence,threat,goals_scored,assists,minutes,own_goals,yellow_cards,red_cards,penalties_missed,selected_by_percent,now_cost,points_per_game,total_points,decision
30,Adam Federici,Adam,Federici,1,GKP,31,Bournemouth,91,Federici,5,0,1,3,1,37,0.0,1.2,34.4,0.0,0,0,180,0,1,0,0,5.6,40,4.0,8,0.0
419,Boaz Myhill,Boaz,Myhill,1,GKP,420,West Brom,35,Myhill,0,0,0,0,0,0,0.0,1.2,0.0,0.0,0,0,0,0,0,0,0,4.8,40,0.0,0,0.0
140,Matthew Pennington,Matthew,Pennington,2,DEF,141,Everton,11,Pennington,0,0,0,6,0,37,10.7,0.0,63.8,33.0,1,0,180,0,0,0,0,0.2,40,3.0,9,0.0
59,Liam Rosenior,Liam,Rosenior,2,DEF,60,Brighton,36,Rosenior,0,0,0,0,0,0,0.0,0.3,0.0,0.0,0,0,0,0,0,0,0,3.3,40,0.0,0,0.0
464,Massadio Haidara,Massadio,Haidara,2,DEF,465,Newcastle,4,Haidara,0,0,0,0,0,0,0.0,-0.2,0.0,0.0,0,0,0,0,0,0,0,1.8,40,0.0,0,0.0
352,Angel Rangel,Angel,Rangel,2,DEF,353,Swansea,80,Rangel,0,0,1,24,1,190,122.7,0.8,221.8,54.0,1,0,831,0,1,0,0,21.8,40,1.4,26,0.0
36,Bradley Smith,Bradley,Smith,2,DEF,37,Bournemouth,91,Brad Smith,0,0,0,9,0,35,30.3,1.3,28.6,27.0,0,0,269,0,1,0,0,4.4,40,0.8,4,0.0
292,JesÃºs GÃ¡mez Duarte,JesÃºs,GÃ¡mez Duarte,2,DEF,293,Newcastle,4,GÃ¡mez,0,0,0,0,0,0,0.0,-0.2,0.0,0.0,0,0,0,0,0,0,0,2.3,40,0.0,0,0.0
350,Kristoffer Nordfeldt,Kristoffer,Nordfeldt,1,GKP,351,Swansea,80,Nordfeldt,4,0,0,1,0,12,0.0,0.7,24.8,0.0,0,0,90,0,0,0,0,1.5,40,3.0,3,0.0
161,Joel Coleman,Joel,Coleman,1,GKP,162,Huddersfield,38,Coleman,0,0,0,0,0,0,0.0,1.2,0.0,0.0,0,0,0,0,0,0,0,2.2,40,0.0,0,0.0


In [234]:
pdata[pdata.decision==1].total_points.sum()

2422